In [77]:
import numpy as np
import collections
import pandas as pd
# for cleaning
import regex as re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from wordcloud import WordCloud
from sklearn.decomposition import PCA
# for tokenizing
from nltk.tokenize import word_tokenize
# vectorize tf-idf
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
# for plots and visuals
import matplotlib.pyplot as plt
import seaborn as sns
import time
# distance eucledian
from scipy.spatial import distance
from sklearn.metrics import pairwise_distances
# reading file form data directory
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# get names of all the files to be clustered
files = os.listdir('/mnt/c/_programming/medius-intern-challenge/ClusterDocs/data')
# ignore the .DS_STORE file
files.remove(files[0])
# store the contents into a list
docsList = []
path = '/mnt/c/_programming/medius-intern-challenge/ClusterDocs/data/'
for file in files:
    p = path+file
    # to handle different encodings
    f = open(p, encoding="ascii", errors="surrogateescape")
    contents = f.read()
    docsList.append(contents)

In [3]:
# parse emails into sender, receiver and body key-value pairs
def parseRawMessage(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'path']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email
def parseEmails(messages):
    emails = [parseRawMessage(message) for message in messages]
    mapp= {'body':[], 'to':[],'_from':[]}
    for email in emails:
        mapp['body'].append(email['body'])
        mapp['to'].append(email['path'])
        mapp['_from'].append(email['from'])
    return mapp

In [4]:
emailDf = pd.DataFrame(parseEmails(docsList))

In [5]:
# removing empty columns if any
emailDf.drop(emailDf.query("body == '' | to == '' | _from == ''").index, inplace=True)

In [14]:
# process the body data by removing all punctuations and stop words, then lemmatizing
def cleanText(docsList):
    cleanDocs = []
    # regex patterns for letters, digits, unedrscores
    p1 = re.compile(r'^\W+|\W+$')
    # pattern for newline, tabs, spaces
    p2 = re.compile(r'\s')
    # pattern for punctuations
    p3 = re.compile(r'[^a-zA-Z0-9]')
    lemma = WordNetLemmatizer()
    
    for doc in docsList:
        newToken = ""
        for word in doc.split():
            if (not p1.match(word)) and (not p2.match(word)) and (not p3.match(word)) and (word not in stopwords.words('english')+['the']):
                newToken = newToken + " " + word.lower()
        newToken = lemma.lemmatize(newToken)
        cleanDocs.append(newToken)
    return cleanDocs

In [15]:
newBody = cleanText(emailDf['body'])
emailDf['body'] = newBody

In [16]:
data = emailDf['body']

tf_idf_vectorizor = TfidfVectorizer(max_features = 5000)
tf_idf = tf_idf_vectorizor.fit_transform(data)

In [65]:
X = tf_idf.todense()


numpy.matrix

In [82]:
class KmeanClass:
    
    def __init__(self, k, max_iter = 200):
        self.k = k
        self.max_iter = max_iter
        
    #Randomly Initialise Centroids
    def init_centroids(self, data):
        initial_centroids = np.random.permutation(data.shape[0])[:self.k]
        self.centroids = data[initial_centroids]

        return self.centroids
    
    # calc distances from each centroid
    def calc_distance(self, data):
        d = []
        for c in self.centroids:
            tmp = distance.euclidean(data, c)
            d.append(tmp)
        return d
    
    #computer distance of data from centroids and assign to min value
    def assign_clusters(self, data): 
        
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        
        dist_to_centroid =  calc_distance(data)
        self.cluster_labels = np.argmin(dist_to_centroid, axis = 1)
        
        return  self.cluster_labels
    
    #Computes average of all data points in cluster and assigns new centroids as average of data points
    def update_centroids(self, data):
        self.centroids = np.array()
        for i in range (self.k):
            
            self.centroids.append(data)
        return self.centroids
    
    #Predict which cluster data point belongs to
    def predict(self, data):
        return self.assign_clusters(data)
    
    #fit data according to the kmeans algo
    def fit(self, data):
        
        self.centroids = self.init_centroids(data)
        
        for i in range(0,self.max_iter):
            self.cluster_labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)
            # we exit if the centroids were not updated after any iteration 
            if self.centroids = old_centroids:
                break
            old_centroids = self.centroids
        return self

In [83]:
model = KmeanClass(k=2).fit(tf_idf)

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.